<a href="https://colab.research.google.com/github/Anurag38/NLP-Practicals/blob/main/Practical%2010%20-%20Text%20Processing%20with%20LSTM/Text_Processing_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv('/content/SMSSpamCollection', sep = '\t', names = ['label','message'])
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
text = data['message']
class_label = data['label']

In [ ]:
import numpy as np
from keras.utils.np_utils import to_categorical
classes_list = ["ham","spam"]
label_index = class_label.apply(classes_list.index)
label1 = np.asarray(label_index)
label = to_categorical(np.asarray(label1))

In [ ]:
label

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
from keras.preprocessing.text import Tokenizer
tk=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tk.fit_on_texts(text)
index=tk.word_index
#print(index)
x = tk.texts_to_sequences(text)
#print (x)
vocab_size = len(index)

In [ ]:
print(vocab_size+1)

9010


In [ ]:
index

{'i': 1,
 'to': 2,
 'you': 3,
 'a': 4,
 'the': 5,
 'u': 6,
 'and': 7,
 'in': 8,
 'is': 9,
 'me': 10,
 'my': 11,
 'for': 12,
 'your': 13,
 'it': 14,
 'of': 15,
 'call': 16,
 'have': 17,
 'on': 18,
 '2': 19,
 'that': 20,
 'now': 21,
 'are': 22,
 'so': 23,
 'but': 24,
 'not': 25,
 'or': 26,
 'do': 27,
 'can': 28,
 'at': 29,
 "i'm": 30,
 'ur': 31,
 'get': 32,
 'will': 33,
 'if': 34,
 'be': 35,
 'with': 36,
 'just': 37,
 'no': 38,
 'we': 39,
 'this': 40,
 '4': 41,
 'gt': 42,
 'lt': 43,
 'up': 44,
 'when': 45,
 'ok': 46,
 'free': 47,
 'from': 48,
 'go': 49,
 'how': 50,
 'all': 51,
 'out': 52,
 'what': 53,
 'know': 54,
 'like': 55,
 'good': 56,
 'then': 57,
 'got': 58,
 'come': 59,
 'was': 60,
 'its': 61,
 'am': 62,
 'time': 63,
 'only': 64,
 'day': 65,
 'love': 66,
 'there': 67,
 'send': 68,
 'he': 69,
 'want': 70,
 'text': 71,
 'as': 72,
 'txt': 73,
 'one': 74,
 'going': 75,
 'by': 76,
 'ü': 77,
 "i'll": 78,
 'need': 79,
 'home': 80,
 'about': 81,
 'r': 82,
 'lor': 83,
 'sorry': 84,
 'stop'

In [ ]:
from keras.preprocessing.sequence import pad_sequences
embedding_vecor_length =32
padded_docs = pad_sequences(x, maxlen=embedding_vecor_length, padding='post')
print (padded_docs)

[[  49  471 4436 ...    0    0    0]
 [  46  336 1499 ...    0    0    0]
 [  47  489    8 ...  392 2998    0]
 ...
 [9007   60    8 ...    0    0    0]
 [   5  534  114 ...    0    0    0]
 [2687   61  465 ...    0    0    0]]


In [ ]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
padded_docs[0]

array([  49,  471, 4436,  842,  755,  658,   64,    8, 1327,   88,  123,
        351, 1328,  148, 2997, 1329,   67,   58, 4437,  144,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, label, test_size=0.30, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Embedding, LSTM, Bidirectional, SimpleRNN, GRU
from keras.models import Sequential, Model

lstm_input= Input(shape=(embedding_vecor_length,),  dtype='int32', name='lstm_input')
x= Embedding(vocab_size+1, 100, input_length=embedding_vecor_length,trainable=True)(lstm_input)
x1=LSTM(256,return_sequences=True)(x)
lstm_out= LSTM(128,return_sequences=False)(x1)
main_output = Dense(2,activation='softmax', name='main_output')(lstm_out)
model = Model(inputs=lstm_input, outputs=main_output)
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
print(model.summary())
#params = 4 * ((size_of_input + 1) * size_of_output + size_of_output^2)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 32)]              0         
                                                                 
 embedding (Embedding)       (None, 32, 100)           901000    
                                                                 
 lstm (LSTM)                 (None, 32, 256)           365568    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 main_output (Dense)         (None, 2)                 258       
                                                                 
Total params: 1,463,946
Trainable params: 1,463,946
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),
              epochs=2,batch_size=100, verbose=2)

Epoch 1/2
39/39 - 18s - loss: 0.2254 - accuracy: 0.9297 - val_loss: 0.0684 - val_accuracy: 0.9791 - 18s/epoch - 456ms/step
Epoch 2/2
39/39 - 14s - loss: 0.0431 - accuracy: 0.9887 - val_loss: 0.0790 - val_accuracy: 0.9839 - 14s/epoch - 356ms/step


In [ ]:
predictions_test = model.predict(X_test)
predictions_test1 = np.zeros_like(predictions_test)
predictions_test1[np.arange(len(predictions_test)), predictions_test.argmax(1)] = 1

In [ ]:
len(X_test)

1672

In [ ]:
predictions_test

array([[9.9999928e-01, 7.1793175e-07],
       [9.9999952e-01, 5.1443266e-07],
       [9.9999952e-01, 4.5062023e-07],
       ...,
       [9.9999952e-01, 4.1849088e-07],
       [1.3468941e-05, 9.9998653e-01],
       [9.9999952e-01, 4.5179152e-07]], dtype=float32)

In [ ]:
predictions_test1

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics
predictions_test1 = np.argmax(predictions_test1,axis=1)

y_test = np.argmax(y_test,axis=1)

print(metrics.confusion_matrix(y_test,predictions_test1))

print(classification_report(y_test,predictions_test1))

[[1448    0]
 [  27  197]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      1448
           1       1.00      0.88      0.94       224

    accuracy                           0.98      1672
   macro avg       0.99      0.94      0.96      1672
weighted avg       0.98      0.98      0.98      1672

